Yes, it is possible to implement an exponential smoothing recurrent neural network (RNN) using PyTorch. Exponential smoothing is a technique that is commonly used in time series forecasting to smooth out short-term fluctuations and make it easier to predict future values.

To implement an exponential smoothing RNN in PyTorch, you can use the torch.nn.Linear and torch.nn.RNN modules, which provide implementations of the linear and recurrent layers, respectively. You can also use the torch.nn.functional.exponential_smooth function to apply exponential smoothing to the output of the RNN. Here is an example of how you might use these modules and functions to define an exponential smoothing RNN model:

In [10]:
import torch

class SMAPELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, Y_true, Y_pred):
        # Compute the absolute difference between Y_true and Y_pred
        diff = torch.abs(Y_true - Y_pred)

        # Compute the absolute values of Y_true and Y_pred
        true = torch.abs(Y_true)
        pred = torch.abs(Y_pred)

        # Compute the sum of the absolute differences divided by the sum of the absolute values
        smape = torch.sum(diff / (true + pred))

        # Multiply by 100 and divide by the number of samples to get the final loss
        return 100 * smape / Y_true.size(0)

In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import pandas as pd
import numpy as np
# df=pd.read_csv('run-of-river_production_load.csv',index_col='Date_Time',parse_dates=True)
df=pd.read_csv('run-of-river_production_load.csv',parse_dates=True)
df["Date_Time"] = pd.to_datetime(df["Date_Time"])
df["date"]=df["Date_Time"].dt.date
df["month"]=df["Date_Time"].dt.month
df["day"]=df["Date_Time"].dt.day
df["hour"]=df["Date_Time"].dt.hour

torch.Tensor

In [12]:
input=pd.DataFrame()
target=pd.DataFrame()
input['month']=df['month']
input['day']=df['day']
input['hour']=df['hour']
target['Value']=df['Value']-df["Value"].min()
input_tonumpy=input.to_numpy()
input_tensor = torch.from_numpy(input_tonumpy).to(torch.float32)
target_tonumpy=target.to_numpy()
target_tensor = torch.from_numpy(target_tonumpy).to(torch.float32)

In [130]:
target_tensor

tensor([[1986.],
        [1984.],
        [1934.],
        ...,
        [1559.],
        [1560.],
        [1559.]])

In [16]:
input_tensor

tensor([[ 1.,  1.,  0.],
        [ 1.,  1.,  0.],
        [ 1.,  1.,  0.],
        ...,
        [ 5.,  1., 23.],
        [ 5.,  1., 23.],
        [ 5.,  1., 23.]])

In [13]:
target_tensor[1]

tensor([1984.])

In [14]:
import torch
from torch import nn

# Define hyperparameters
learning_rate = 0.001
smoothing_constant = 0.5

# Define the recurrent neural network
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.prev_input=1
        
    def forward(self, input):
    # Use exponential smoothing on the input
        smoothed_input = input * smoothing_constant + (1 - smoothing_constant) * self.prev_input
        self.prev_input = smoothed_input

    # Pass the smoothed input through the LSTM layer
        lstm_out, hidden = self.lstm(smoothed_input.view(len(input), 1, -1))

    # Pass the LSTM output through the fully-connected layer
        output = self.fc(lstm_out.view(len(input), -1))
        return output
# Initialize the model, loss function, and optimizer
model = RNN(1, 128, 1)
# criterion = nn.MSELoss()
criterion = SMAPELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Train the model
num_epochs=30
for epoch in range(num_epochs):
    for i in range(len(input_tensor)):
        # Make predictions using the model
        output = model.forward(input_tensor[i])
    
        # Calculate the loss
        loss = criterion(output, target_tensor[i])
        #loss = smape.loss(output, target_tensor[i])
        # Update the model's parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}: Loss = {loss.item():.4f}')
#     print(f'Epoch {epoch+1}: SMAPE = {loss.item():.4f}')
#         print(input_tensor[i].type())


Epoch 1: Loss = 8.4014
Epoch 2: Loss = 8.4148
Epoch 3: Loss = 8.1446
Epoch 4: Loss = 8.0112
Epoch 5: Loss = 8.1470
Epoch 6: Loss = 8.4425
Epoch 7: Loss = 7.9066
Epoch 8: Loss = 7.1465
Epoch 9: Loss = 7.1472
Epoch 10: Loss = 7.3415
Epoch 11: Loss = 7.2435
Epoch 12: Loss = 7.0839
Epoch 13: Loss = 7.4909
Epoch 14: Loss = 7.8744
Epoch 15: Loss = 7.3675
Epoch 16: Loss = 7.3687
Epoch 17: Loss = 7.4036
Epoch 18: Loss = 7.3290
Epoch 19: Loss = 7.1996
Epoch 20: Loss = 6.8254
Epoch 21: Loss = 6.7529
Epoch 22: Loss = 6.4671
Epoch 23: Loss = 7.0538
Epoch 24: Loss = 6.7941
Epoch 25: Loss = 6.9835
Epoch 26: Loss = 6.4110
Epoch 27: Loss = 5.9486
Epoch 28: Loss = 6.6391
Epoch 29: Loss = 6.1255
Epoch 30: Loss = 6.1857


In [133]:
output

tensor([[1059.0864],
        [1081.5814],
        [1085.0291]], grad_fn=<AddmmBackward0>)